# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import sys
sys.executable

'C:\\Users\\tete1\\miniconda3\\envs\\jupyter_env\\python.exe'

In [2]:
!pip install scipy

  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/fd/a7/5f829b100d208c85163aecba93faf01d088d944fc91585338751d812f1e4/scipy-1.12.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.4 kB 330.3 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.4 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 534.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/46.2 MB 10.2 MB/s eta 0:00:05
    --------------------------------------- 1.0/46.2 MB 12.1 MB/s eta 0:00:04
   - -------------------------------------- 2.0/46.2 MB 15.8 MB/s eta 0:00:03
   -- ------------------------------------- 3.4/46.2 MB 19.5 MB/s eta 0:00:03
   ---- ----------------------------------- 5.2/46.2 MB 23.6 MB/s eta 0:00:02
   ----- --------------

In [3]:
import sys
sys.version

'3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]'

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
## Paso 1 groupby (customer_id, product_name) >> sum(quantity)

In [6]:
ventas = data.groupby(by=['CustomerID', 'ProductName'])["Quantity"].sum().reset_index()

In [7]:
ventas.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [8]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63628 entries, 0 to 63627
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   63628 non-null  int64 
 1   ProductName  63628 non-null  object
 2   Quantity     63628 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [9]:
## pivot pable-> df, values, index, columns + fillna a 0 y fillvalues a 0

In [10]:
pivot_t = ventas.pivot_table(values='Quantity', index='ProductName', columns='CustomerID', fill_value=0)

In [11]:
pivot_t

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [12]:
dist_calculation = pdist(X=pivot_t, metric='euclidean')

In [13]:
intermedio = pivot_t.T   # Pendiente investigar el paso intermedio

In [14]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(intermedio, 'euclidean'))),
                                index=pivot_t.columns,
                                columns=pivot_t.columns)

euclid_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [15]:
'''# Ejemplo talk 

# Frame of reference

euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(users_ratings, 'euclidean'))),
                                index=movie_ratings.columns,
                                columns=movie_ratings.columns)

euclid_dist_norm'''

"# Ejemplo talk \n\n# Frame of reference\n\neuclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(users_ratings, 'euclidean'))),\n                                index=movie_ratings.columns,\n                                columns=movie_ratings.columns)\n\neuclid_dist_norm"

In [16]:
# Documentación: 
# Motores de recomendación parte 1: https://www.datahack.es/motores-de-recomendacion-con-python-parte-1/ 
# Motores de recomendación parte 2 (texto): https://www.datahack.es/motores-de-recomendacion-con-python-parte-2

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [17]:
type(euclid_dist_norm)

pandas.core.frame.DataFrame

In [18]:
resultado = euclid_dist_norm.sort_values(by = 33, ascending=False)  # Ordenas por el cliente 33 y en orden ascendente

In [19]:
resultado = resultado[33]  # Seleccionas el customerID de 33

In [20]:
resultado = resultado.head(5)

In [21]:
type(resultado)

pandas.core.series.Series

In [53]:
resultado

CustomerID
33      1.000000
3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [ ]:
# df(step1) + df(step4 - customerid) >> merge / join  >> df_final(customerid, customer_name....)

In [22]:
ventas

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


In [26]:
resultado

CustomerID
33      1.000000
3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
Name: 33, dtype: float64

In [30]:
df_result = pd.DataFrame(resultado).reset_index()

In [39]:
df_result.columns.values[0]

'CustomerID'

In [40]:
step5 = ventas[ventas['CustomerID'].isin(df_result['CustomerID'])]

In [44]:
step5

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
# groupby (productName) >> sum(productname + quantity) >> sort (ascending=False)

In [47]:
step6 = step5.groupby(by=['ProductName'])["Quantity"].sum().reset_index().sort_values(by='Quantity', ascending=False)

In [48]:
step6       # DF de productos rankeados

,ProductName,Quantity
134,"Pepper - Black, Whole",4
38,Butter - Unsalted,3
89,Grouper - Fresh,3
191,"Veal - Inside, Choice",3
193,Veal - Sweetbread,3
...,...,...
78,Fenngreek Seed,1
79,"Fish - Scallops, Cold Smoked",1
80,Flour - Teff,1
81,Foam Dinner Plate,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [49]:
pivot_t

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


In [51]:
step7 = pd.merge(step6, pivot_t, left_on='ProductName', right_index=True)
# Join por índice de filas:https://interactivechaos.com/es/manual/tutorial-de-pandas/join-por-indice-de-filas

In [52]:
step7

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
134,"Pepper - Black, Whole",4,1,1,0,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
38,Butter - Unsalted,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
89,Grouper - Fresh,3,2,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191,"Veal - Inside, Choice",3,2,0,0,0,1,0,0,1,...,0,0,0,0,25,0,0,0,0,0
193,Veal - Sweetbread,3,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Fenngreek Seed,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
79,"Fish - Scallops, Cold Smoked",1,0,0,0,0,1,0,1,0,...,0,0,0,25,25,0,0,0,0,0
80,Flour - Teff,1,0,0,1,0,0,0,0,0,...,25,0,0,0,50,25,0,25,0,0
81,Foam Dinner Plate,1,0,0,0,0,1,0,0,0,...,25,0,0,0,0,0,0,0,0,0


In [54]:
df_result['CustomerID']

0      33
1    3317
2    3535
3     264
4    2503
Name: CustomerID, dtype: int64

In [ ]:
# filter (records == 0) >> sort 5

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [5]:
# Pide una lista que contenga diccionarios [{}, {}, {}] >> para completarla bucle for

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.